In [1]:
pip install transformers

     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
     |████████████████████████████████| 636 kB 47.7 MB/s 
     |████████████████████████████████| 3.3 MB 57.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.3 MB/s 


In [3]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer, XLMRobertaForSequenceClassification, XLMRobertaTokenizer, GPT2ForSequenceClassification, PreTrainedTokenizer, PreTrainedTokenizerFast, BertForSequenceClassification, BertTokenizer
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn 
import torch 
import torch.nn as nn 
import torch.functional as f 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
import time 
import datetime
import torch.nn.functional as nnf


# Load Data

In [4]:
device = torch.device('cuda')
BATCH_SIZE = 8
NUM_EPOCHS = 20
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA, GPT2

In [12]:
test = pd.read_csv('./drive/MyDrive/dacon_NLP/nlp_test.csv') 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] 
test['요약문_내용'].fillna('NAN',inplace=True)
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean


def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total


In [7]:
## load XLM-Roberta 
roberta_checkpoint = torch.load("./drive/MyDrive/dacon_NLP/XLM_ROBERTA_BASE_voting_6")
test_roberta = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
test_roberta.load_state_dict(roberta_checkpoint) 
test_roberta.cuda()
print() 

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [8]:
tokenizer_roberta = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_roberta.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask


In [9]:
tokenizer_roberta 

PreTrainedTokenizer(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [12]:
roberta_probsum = [] 
roberta_chunkparts = [] 

# change to eval mode 
test_roberta.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_roberta = [] # store probabilities for electra 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + f6 + " " + text 
        # tokenize test text 
        input_id, attention_mask = roberta_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        with torch.no_grad(): 
            outputs = test_roberta(input_id, 
                                   token_type_ids=None,
                                   attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_roberta.append(prob)  

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_roberta)):  
      for j in range(46): 
        prob_sum[j] += probabilities_roberta[i][j]   
    
    roberta_probsum.append(prob_sum) 
    roberta_chunkparts.append(len(probabilities_roberta))


roberta_probsum = np.asarray(roberta_probsum) 
roberta_chunkparts = np.asarray(roberta_chunkparts)

print("saving predicted probabilities values") 
np.save('./drive/MyDrive/dacon_NLP/roberta_probsum.npy', roberta_probsum) 
np.save('./drive/MyDrive/dacon_NLP/roberta_chunkparts.npy', roberta_chunkparts)


  7%|▋         | 3141/43576 [11:15<2:21:51,  4.75it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [30:57<2:22:48,  4.08it/s]

Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [1:36:46<47:31,  5.75it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [2:35:10<00:00,  4.68it/s]

saving predicted probabilities values


In [13]:
# coding=utf-8
# Copyright 2021 SKT AI Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from typing import Any, Dict, List, Optional
from transformers.tokenization_utils import AddedToken
from transformers import XLNetTokenizer


class KoBERTTokenizer(XLNetTokenizer):
    def __init__(
        self,
        vocab_file,
        do_lower_case=False,
        remove_space=True,
        keep_accents=False,
        bos_token="[CLS]",
        eos_token="[SEP]",
        unk_token="[UNK]",
        sep_token="[SEP]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        mask_token="[MASK]",
        additional_special_tokens=None,
        sp_model_kwargs: Optional[Dict[str, Any]] = None,
        **kwargs
    ) -> None:
        # Mask token behave like a normal word, i.e. include the space before it
        mask_token = AddedToken(mask_token, lstrip=True, rstrip=False) if isinstance(mask_token, str) else mask_token

        self.sp_model_kwargs = {} if sp_model_kwargs is None else sp_model_kwargs

        super().__init__(
            vocab_file,
            do_lower_case=do_lower_case,
            remove_space=remove_space,
            keep_accents=keep_accents,
            bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            additional_special_tokens=additional_special_tokens,
            sp_model_kwargs=self.sp_model_kwargs,
            **kwargs,
        )
    
    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks by concatenating and
        adding special tokens. An XLNet sequence has the following format:
        - single sequence: ``<cls> X <sep>``
        - pair of sequences: ``<cls> A <sep> B <sep>``
        Args:
            token_ids_0 (:obj:`List[int]`):
                List of IDs to which the special tokens will be added.
            token_ids_1 (:obj:`List[int]`, `optional`):
                Optional second list of IDs for sequence pairs.
        Returns:
            :obj:`List[int]`: List of `input IDs <../glossary.html#input-ids>`__ with the appropriate special tokens.
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return cls + token_ids_0 + sep
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def _tokenize(self, text: str) -> List[str]:
        """Tokenize a string."""
        text = self.preprocess_text(text)
        pieces = self.sp_model.encode(text, out_type=str, **self.sp_model_kwargs)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def create_token_type_ids_from_sequences(
            self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
        ) -> List[int]:
            """
            Create a mask from the two sequences passed to be used in a sequence-pair classification task. An XLNet
            sequence pair mask has the following format:
            ::
                0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
                | first sequence    | second sequence |
            If :obj:`token_ids_1` is :obj:`None`, this method only returns the first portion of the mask (0s).
            Args:
                token_ids_0 (:obj:`List[int]`):
                    List of IDs.
                token_ids_1 (:obj:`List[int]`, `optional`):
                    Optional second list of IDs for sequence pairs.
            Returns:
                :obj:`List[int]`: List of `token type IDs <../glossary.html#token-type-ids>`_ according to the given
                sequence(s).
            """
            sep = [self.sep_token_id]
            cls = [self.cls_token_id]
            if token_ids_1 is None:
                return len(cls + token_ids_0 + sep) * [0]
            return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]


In [14]:
# Load KoBERT
bert_checkpoint = torch.load("drive/MyDrive/dacon_NLP/KoBERT_MORE_FEATURES_6")
test_bert = BertForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=46)
test_bert.load_state_dict(bert_checkpoint)
test_bert.cuda()
print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#from kobert_tokenizer import KoBERTTokenizer
tokenizer_bert = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')


def bert_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_bert.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [18]:
bert_probsum = [] 
bert_chunkparts = [] 

# change to eval mode 
test_bert.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    #f6 = str(test_feature6[i]) 
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_bert = [] # store probabilities for electra 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        # tokenize test text 
        input_id, attention_mask, token_type_ids = bert_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_ids = torch.tensor(token_type_ids) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        token_type_ids = torch.reshape(token_type_ids, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_ids = token_type_ids.to(device) 
        with torch.no_grad(): 
            outputs = test_bert(input_id, 
                                token_type_ids=token_type_ids,
                                attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_bert.append(prob)  

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_bert)):  
      for j in range(46): 
        prob_sum[j] += probabilities_bert[i][j]   
    
    bert_probsum.append(prob_sum) 
    bert_chunkparts.append(len(probabilities_bert))


bert_probsum = np.asarray(bert_probsum) 
bert_chunkparts = np.asarray(bert_chunkparts)

print("saving predicted probabilities values") 
np.save('./drive/MyDrive/dacon_NLP/bert_probsum.npy', bert_probsum) 
np.save('./drive/MyDrive/dacon_NLP/bert_chunkparts.npy', bert_chunkparts)


  7%|▋         | 3141/43576 [11:13<2:21:52,  4.75it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [30:51<2:22:01,  4.11it/s]

Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [1:36:21<47:06,  5.80it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [2:34:33<00:00,  4.70it/s]

saving predicted probabilities values


# GPT-2

In [6]:
tokenizer_gpt2 = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                     bos_token='</s>',
                                                     eos_token='</s>',
                                                     unk_token='<unk>',
                                                     pad_token='<pad>',
                                                     mask_token='<mask>') 

 

def gpt2_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_gpt2.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [7]:
## load gpt2 
gpt2_checkpoint = torch.load("./drive/MyDrive/dacon_NLP/KoGPT2_voting_7")
test_gpt2 = GPT2ForSequenceClassification.from_pretrained("skt/kogpt2-base-v2", num_labels=46) 
test_gpt2.load_state_dict(gpt2_checkpoint)
test_gpt2.cuda() 
print() 

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenizer_gpt2.padding_side = 'left'
test_gpt2.resize_token_embeddings(len(tokenizer_gpt2))
test_gpt2.config.pad_token_id = tokenizer_gpt2.pad_token_id


In [13]:
gpt2_probsum = [] 
gpt2_chunkparts = [] 

# change to eval mode 
test_gpt2.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_gpt2 = [] # store probabilities for electra 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        # tokenize test text 
        input_id, attention_mask = gpt2_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        with torch.no_grad(): 
            outputs = test_gpt2(input_id, 
                                token_type_ids=None,
                                attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_gpt2.append(prob)  

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_gpt2)):  
      for j in range(46): 
        prob_sum[j] += probabilities_gpt2[i][j]   
    
    gpt2_probsum.append(prob_sum) 
    gpt2_chunkparts.append(len(probabilities_gpt2))


gpt2_probsum = np.asarray(gpt2_probsum) 
gpt2_chunkparts = np.asarray(gpt2_chunkparts)

print("saving predicted probabilities values") 
np.save('./drive/MyDrive/dacon_NLP/gpt2_probsum.npy', gpt2_probsum) 
np.save('./drive/MyDrive/dacon_NLP/gpt2_chunkparts.npy', gpt2_chunkparts)


  7%|▋         | 3141/43576 [13:36<2:52:48,  3.90it/s]

Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [37:25<2:55:06,  3.33it/s]

Long Text!! Using Head+Tail Truncation


 60%|██████    | 26322/43576 [1:53:21<2:21:23,  2.03it/s]

...

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [1:56:56<55:18,  4.94it/s]  

Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [3:07:12<00:00,  3.88it/s]

saving predicted probabilities values


# ELECTRA

In [14]:
electra_checkpoint = torch.load('drive/MyDrive/dacon_NLP/ELECTRA_MORE_FEATURES_6') 
test_electra = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=46) 
test_electra.load_state_dict(electra_checkpoint) 
test_electra.cuda()
print()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [15]:
tokenizer_electra = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 

def electra_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer_electra.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences, not "really" necessary for now    
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        token_type_id = token_type_id[:129] + token_type_id[-383:]    
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        token_type_id = token_type_id + [0]*(512 - len(token_type_id))  
        
    return input_id, attention_mask, token_type_id


In [16]:
electra_probsum = [] 
electra_chunkparts = [] 

# change to eval mode 
test_electra.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    splitted = split_text(clean_text(str(test_contents[i]))) 
    # make predictions for each splitted text 
    probabilities_electra = [] # store probabilities for electra 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + text 
        # tokenize test text 
        input_id, attention_mask, token_type_ids = electra_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        token_type_ids = torch.tensor(token_type_ids) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1, MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1, MAX_LEN)) 
        token_type_ids = torch.reshape(token_type_ids, (-1, MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        token_type_ids = token_type_ids.to(device) 
        with torch.no_grad(): 
            outputs = test_electra(input_id, 
                                   token_type_ids=token_type_ids,
                                   attention_mask = attention_mask)

        logits = outputs[0]
        
        # obtain softmax probabilities  
        prob = nnf.softmax(logits, dim=1).flatten() 
        probabilities_electra.append(prob)  

    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities_electra)):  
      for j in range(46): 
        prob_sum[j] += probabilities_electra[i][j]   
    
    electra_probsum.append(prob_sum) 
    electra_chunkparts.append(len(probabilities_electra))


electra_probsum = np.asarray(electra_probsum) 
electra_chunkparts = np.asarray(electra_chunkparts)

print("saving predicted probabilities values") 
np.save('./drive/MyDrive/dacon_NLP/electra_probsum.npy', electra_probsum) 
np.save('./drive/MyDrive/dacon_NLP/electra_chunkparts.npy', electra_chunkparts)


  7%|▋         | 3141/43576 [11:47<2:31:14,  4.46it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 20%|█▉        | 8594/43576 [32:26<2:28:54,  3.92it/s]

Long Text!! Using Head+Tail Truncation


 29%|██▊       | 12438/43576 [46:48<1:44:19,  4.97it/s]

...

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 62%|██████▏   | 27182/43576 [1:41:18<49:33,  5.51it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


100%|██████████| 43576/43576 [2:42:42<00:00,  4.46it/s]

saving predicted probabilities values
